In [1]:
import ipywidgets.widgets as widgets
from jetbot import Robot, Camera, bgr8_to_jpeg, Heartbeat
import traitlets
import os
import ipywidgets as widgets
from IPython.display import display
import traitlets
from jetbot import Camera, bgr8_to_jpeg
from datetime import datetime

# establish a connection with the controller
controller = widgets.Controller(index=0) # change index if necessary
display(controller)

Error loading module `ublox_gps`: No module named 'serial'


Controller()

In [2]:
# link controller to motors
robot = Robot()

left_link = traitlets.dlink((controller.axes[1], 'value'), (robot.left_motor, 'value'), transform=lambda x: -x)
right_link = traitlets.dlink((controller.axes[3], 'value'), (robot.right_motor, 'value'), transform=lambda x: -x)

# establish video feed
image = widgets.Image(format='jpeg', width=300, height=300)
camera = Camera.instance()
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

# create killswitch function that ceases the JetBot's functions if disconnected
def handle_heartbeat_status(change):
    if change['new'] == Heartbeat.Status.dead:
        camera_link.unlink()
        left_link.unlink()
        right_link.unlink()
        robot.stop()

heartbeat = Heartbeat(period=0.5)
heartbeat.observe(handle_heartbeat_status, names='status')

In [3]:
# global state
run_initialized = False
run_dirs = {}

# initialize directory structure
def init_run_dirs():
    global run_initialized, run_dirs
    suffix = suffix_input.value.strip()
    if not suffix:
        print("❌ Please enter a suffix before saving.")
        return False

    base_dir = os.path.join('./data', suffix)
    labels = ['left', 'forward', 'right']
    run_dirs = {}

    for label in labels:
        subfolder = f"{label}_{suffix}"
        full_path = os.path.join(base_dir, subfolder)
        os.makedirs(full_path, exist_ok=True)
        run_dirs[label] = full_path

    run_initialized = True
    print(f"✅ Initialized run folders under: {base_dir}")
    return True

# save snapshot
def save_snapshot(label):
    if not run_initialized:
        if not init_run_dirs():
            return
    dir_path = run_dirs[label]
    suffix = suffix_input.value.strip()
    filename = f"{label}_{suffix}_{len(os.listdir(dir_path))}.jpg"
    path = os.path.join(dir_path, filename)
    with open(path, 'wb') as f:
        f.write(image.value)

# save and update counter
def save_and_update(label):
    save_snapshot(label)
    count_widgets[label].value = len(os.listdir(run_dirs[label]))
    
# controller bindings
def make_button_handler(label):
    def handler(change):
        if change['new'] == 1.0:
            save_and_update(label)
    return handler

In [4]:
# create filename suffix input
suffix_input = widgets.Text(
    value='',
    placeholder='enter run name (e.g. run1)',
    description='run name:',
    layout=widgets.Layout(width='300px')
)

# widgets
button_layout = widgets.Layout(width='128px', height='64px')
count_widgets = {
    'left': widgets.IntText(layout=button_layout, value=0),
    'forward': widgets.IntText(layout=button_layout, value=0),
    'right': widgets.IntText(layout=button_layout, value=0),
}
buttons = {
    'left': widgets.Button(description='⬅️ (LB)', layout=button_layout),
    'forward': widgets.Button(description='⬆️ (L2/R2)', layout=button_layout),
    'right': widgets.Button(description='➡️ (RB)', layout=button_layout),
}
buttons['left'].on_click(lambda x: save_and_update('left'))
buttons['forward'].on_click(lambda x: save_and_update('forward'))
buttons['right'].on_click(lambda x: save_and_update('right'))

controller.buttons[4].observe(make_button_handler('left'), names='value')
controller.buttons[5].observe(make_button_handler('right'), names='value')
controller.buttons[6].observe(make_button_handler('forward'), names='value')
controller.buttons[7].observe(make_button_handler('forward'), names='value')

In [5]:
# display GUI
display(suffix_input)
display(widgets.HBox([count_widgets['left'], count_widgets['forward'], count_widgets['right']]))
display(widgets.HBox([buttons['left'], buttons['forward'], buttons['right']]))
display(image) # comment out as needed if JetBot keeps freezing

Text(value='', description='Run name:', layout=Layout(width='300px'), placeholder='Enter run name (e.g. run1)'…

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…